<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Complete LangGraph Guide

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

Build stateful, multi-agent workflows with LangGraph and Gemini 3 Pro.

In [ ]:
!pip install -q langgraph langchain langchain-google-genai

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from google.colab import userdata
import operator
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-3-pro", temperature=0.7)

## 1. Simple Graph

In [ ]:
# Define state
class SimpleState(TypedDict):
    messages: Annotated[list, operator.add]
    count: int

# Define nodes
def node_1(state: SimpleState):
    print("Node 1 executing")
    return {
        "messages": ["Node 1 processed"],
        "count": state.get("count", 0) + 1
    }

def node_2(state: SimpleState):
    print("Node 2 executing")
    return {
        "messages": ["Node 2 processed"],
        "count": state["count"] + 1
    }

# Build graph
workflow = StateGraph(SimpleState)
workflow.add_node("node_1", node_1)
workflow.add_node("node_2", node_2)

workflow.set_entry_point("node_1")
workflow.add_edge("node_1", "node_2")
workflow.add_edge("node_2", END)

app = workflow.compile()

# Run
result = app.invoke({"messages": [], "count": 0})
print(f"\nFinal state: {result}")

## 2. Conditional Routing

In [ ]:
class ConditionalState(TypedDict):
    question: str
    answer: str
    needs_search: bool

def analyze_question(state: ConditionalState):
    """Determine if question needs search."""
    question = state["question"]
    needs_search = "latest" in question.lower() or "current" in question.lower()
    return {"needs_search": needs_search}

def search(state: ConditionalState):
    """Simulated search."""
    return {"answer": "Search results: Latest information found."}

def direct_answer(state: ConditionalState):
    """Answer without search."""
    response = llm.predict(state["question"])
    return {"answer": response}

def route(state: ConditionalState):
    """Route based on needs_search."""
    if state.get("needs_search", False):
        return "search"
    return "direct_answer"

# Build graph
workflow = StateGraph(ConditionalState)
workflow.add_node("analyze", analyze_question)
workflow.add_node("search", search)
workflow.add_node("direct_answer", direct_answer)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges(
    "analyze",
    route,
    {
        "search": "search",
        "direct_answer": "direct_answer"
    }
)
workflow.add_edge("search", END)
workflow.add_edge("direct_answer", END)

app = workflow.compile()

# Test
result = app.invoke({"question": "What is AI?", "answer": "", "needs_search": False})
print(f"Answer: {result['answer']}")

## 3. Agent Loop

In [ ]:
from langchain.tools import tool

@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions."""
    try:
        return str(eval(expression))
    except:
        return "Error"

class AgentState(TypedDict):
    messages: list
    next_action: str
    iterations: int

tools = [calculator]

def agent_node(state: AgentState):
    """Agent decides next action."""
    messages = state["messages"]
    last_message = messages[-1] if messages else ""
    
    prompt = f"""
Current task: {last_message}

Available tools: {[t.name for t in tools]}

What should I do next? Reply with:
TOOL: tool_name | input
or
DONE: final answer
"""
    
    response = llm.predict(prompt)
    
    if "DONE:" in response:
        return {
            "next_action": "done",
            "messages": [response.split("DONE:")[1].strip()]
        }
    elif "TOOL:" in response:
        return {
            "next_action": "tool",
            "messages": [response]
        }
    
    return {"next_action": "done", "messages": [response]}

def tool_node(state: AgentState):
    """Execute tool."""
    last_message = state["messages"][-1]
    if "TOOL:" in last_message:
        parts = last_message.split("TOOL:")[1].split("|")
        tool_name = parts[0].strip()
        tool_input = parts[1].strip() if len(parts) > 1 else ""
        
        result = calculator(tool_input)
        return {
            "next_action": "agent",
            "messages": [f"Tool result: {result}"],
            "iterations": state.get("iterations", 0) + 1
        }
    
    return {"next_action": "done"}

def should_continue(state: AgentState):
    if state.get("iterations", 0) > 5:
        return "done"
    return state.get("next_action", "done")

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tool", tool_node)
workflow.add_node("done", lambda x: x)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tool": "tool",
        "done": "done",
        "agent": "agent"
    }
)
workflow.add_edge("tool", "agent")
workflow.add_edge("done", END)

app = workflow.compile()

# Test
result = app.invoke({
    "messages": ["Calculate 25 * 48"],
    "next_action": "",
    "iterations": 0
})
print(f"Result: {result['messages'][-1]}")

## 4. Multi-Agent Workflow

In [ ]:
class MultiAgentState(TypedDict):
    task: str
    research: str
    draft: str
    final: str

def researcher(state: MultiAgentState):
    """Research agent."""
    prompt = f"Research this topic: {state['task']}"
    research = llm.predict(prompt)
    return {"research": research}

def writer(state: MultiAgentState):
    """Writer agent."""
    prompt = f"Write content based on: {state['research']}"
    draft = llm.predict(prompt)
    return {"draft": draft}

def editor(state: MultiAgentState):
    """Editor agent."""
    prompt = f"Edit and improve: {state['draft']}"
    final = llm.predict(prompt)
    return {"final": final}

# Build workflow
workflow = StateGraph(MultiAgentState)
workflow.add_node("researcher", researcher)
workflow.add_node("writer", writer)
workflow.add_node("editor", editor)

workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", "editor")
workflow.add_edge("editor", END)

app = workflow.compile()

# Execute
result = app.invoke({
    "task": "Future of AI",
    "research": "",
    "draft": "",
    "final": ""
})

print(f"Final output: {result['final'][:200]}...")

## Key Takeaways

✅ **LangGraph Features:**
- **State Management**: Shared state across nodes
- **Conditional Routing**: Dynamic flow control
- **Agent Loops**: Iterative reasoning
- **Multi-Agent**: Specialized agent collaboration

📌 **Use Cases:**
- Complex decision workflows
- Multi-step reasoning
- Agent orchestration
- Iterative refinement

🔗 **Resources:**
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI)